# Lab - Random Forests

### Instructions

Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.
Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE 
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [2]:
df_cat=pd.read_csv(r"files_for_lab/categorical.csv")
df_cat.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,87,1,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,86,1,20,1,93,10,96,1,96,1,79,3


In [3]:
df_cat.info() # as we can see, the categorical dataframe has lots of numerical data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STATE         95412 non-null  object
 1   CLUSTER       95412 non-null  int64 
 2   HOMEOWNR      95412 non-null  object
 3   GENDER        95412 non-null  object
 4   DATASRCE      95412 non-null  int64 
 5   RFA_2R        95412 non-null  object
 6   RFA_2A        95412 non-null  object
 7   GEOCODE2      95412 non-null  object
 8   DOMAIN_A      95412 non-null  object
 9   DOMAIN_B      95412 non-null  int64 
 10  ODATEW_YR     95412 non-null  int64 
 11  ODATEW_MM     95412 non-null  int64 
 12  DOB_YR        95412 non-null  int64 
 13  DOB_MM        95412 non-null  int64 
 14  MINRDATE_YR   95412 non-null  int64 
 15  MINRDATE_MM   95412 non-null  int64 
 16  MAXRDATE_YR   95412 non-null  int64 
 17  MAXRDATE_MM   95412 non-null  int64 
 18  LASTDATE_YR   95412 non-null  int64 
 19  LAST

In [4]:
df_num=pd.read_csv(r"files_for_lab/numerical.csv")
df_num.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65,73,59,70,56,78,62,82,99,4,10,5,2,6,12,0,1,9,5,18,20,5,7,6,0,11,33,4,3,2,12,3,3

In [5]:
df_num.info(verbose=True) # in this dataframe all the variables are numeric, either floats or integers

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 315 columns):
 #   Column    Dtype  
---  ------    -----  
 0   TCODE     int64  
 1   AGE       float64
 2   INCOME    int64  
 3   WEALTH1   int64  
 4   HIT       int64  
 5   MALEMILI  int64  
 6   MALEVET   int64  
 7   VIETVETS  int64  
 8   WWIIVETS  int64  
 9   LOCALGOV  int64  
 10  STATEGOV  int64  
 11  FEDGOV    int64  
 12  WEALTH2   int64  
 13  POP901    int64  
 14  POP902    int64  
 15  POP903    int64  
 16  POP90C1   int64  
 17  POP90C2   int64  
 18  POP90C3   int64  
 19  POP90C4   int64  
 20  POP90C5   int64  
 21  ETH1      int64  
 22  ETH2      int64  
 23  ETH3      int64  
 24  ETH4      int64  
 25  ETH5      int64  
 26  ETH6      int64  
 27  ETH7      int64  
 28  ETH8      int64  
 29  ETH9      int64  
 30  ETH10     int64  
 31  ETH11     int64  
 32  ETH12     int64  
 33  ETH13     int64  
 34  ETH14     int64  
 35  ETH15     int64  
 36  ETH16     i

In [6]:
df_num.isnull().sum()

TCODE       0
AGE         0
INCOME      0
WEALTH1     0
HIT         0
           ..
AVGGIFT     0
CONTROLN    0
HPHONE_D    0
RFA_2F      0
CLUSTER2    0
Length: 315, dtype: int64

In [7]:
df_tar=pd.read_csv(r"files_for_lab/target.csv")
df_tar.head()

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [8]:
df_tar.shape, df_num.shape,df_cat.shape

((95412, 2), (95412, 315), (95412, 22))

In [9]:
df_tar['TARGET_B'].value_counts()
    

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [10]:
df_tar['TARGET_D'].value_counts()

0.00      90569
10.00       941
15.00       591
20.00       577
5.00        503
          ...  
101.00        1
4.50          1
55.00         1
10.70         1
13.92         1
Name: TARGET_D, Length: 71, dtype: int64

In [11]:
df_tar['TARGET_D'].describe()

count    95412.000000
mean         0.793073
std          4.429725
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        200.000000
Name: TARGET_D, dtype: float64

In [12]:
df_tar['TARGET_D'].isnull().sum()

0

**We are going to extract the numeric varibles from *df_cat*.**

In [13]:
df_numeric1=df_cat.select_dtypes(include=np.number)

In [14]:
df_numeric1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   CLUSTER       95412 non-null  int64
 1   DATASRCE      95412 non-null  int64
 2   DOMAIN_B      95412 non-null  int64
 3   ODATEW_YR     95412 non-null  int64
 4   ODATEW_MM     95412 non-null  int64
 5   DOB_YR        95412 non-null  int64
 6   DOB_MM        95412 non-null  int64
 7   MINRDATE_YR   95412 non-null  int64
 8   MINRDATE_MM   95412 non-null  int64
 9   MAXRDATE_YR   95412 non-null  int64
 10  MAXRDATE_MM   95412 non-null  int64
 11  LASTDATE_YR   95412 non-null  int64
 12  LASTDATE_MM   95412 non-null  int64
 13  FIRSTDATE_YR  95412 non-null  int64
 14  FIRSTDATE_MM  95412 non-null  int64
dtypes: int64(15)
memory usage: 10.9 MB


**Next we are going to generate a dataframe just with the categorical variables by dropping the previously extracted numeric ones.**

In [15]:
df_categorical1=df_cat.drop(columns=df_numeric1.columns) 

In [16]:
df_categorical1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   STATE     95412 non-null  object
 1   HOMEOWNR  95412 non-null  object
 2   GENDER    95412 non-null  object
 3   RFA_2R    95412 non-null  object
 4   RFA_2A    95412 non-null  object
 5   GEOCODE2  95412 non-null  object
 6   DOMAIN_A  95412 non-null  object
dtypes: object(7)
memory usage: 5.1+ MB


In order to just have a numeric dataframe we are going to concatenate *df_num* and *df_numeric1*.

In [17]:
frames=[df_num,df_numeric1]
df_numeric=pd.concat(frames, axis=1)

In [18]:
df_numeric.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 330 columns):
 #   Column        Dtype  
---  ------        -----  
 0   TCODE         int64  
 1   AGE           float64
 2   INCOME        int64  
 3   WEALTH1       int64  
 4   HIT           int64  
 5   MALEMILI      int64  
 6   MALEVET       int64  
 7   VIETVETS      int64  
 8   WWIIVETS      int64  
 9   LOCALGOV      int64  
 10  STATEGOV      int64  
 11  FEDGOV        int64  
 12  WEALTH2       int64  
 13  POP901        int64  
 14  POP902        int64  
 15  POP903        int64  
 16  POP90C1       int64  
 17  POP90C2       int64  
 18  POP90C3       int64  
 19  POP90C4       int64  
 20  POP90C5       int64  
 21  ETH1          int64  
 22  ETH2          int64  
 23  ETH3          int64  
 24  ETH4          int64  
 25  ETH5          int64  
 26  ETH6          int64  
 27  ETH7          int64  
 28  ETH8          int64  
 29  ETH9          int64  
 30  ETH10         int64  

In [19]:
df_numeric.shape 

(95412, 330)

#Numerical data should be reduced while categorical data has only 7 columns
#Then, use PCA
pca = PCA(n_components=0.95)    
pca.fit(df_numeric, df_tar['TARGET_B'])
df_numeric_pca = pca.transform(df_numeric)
df_numeric_pca.shape

In [20]:
scaler=StandardScaler()

In [43]:
scaled=scaler.fit_transform(df_numeric)

In [44]:
df_num_scaled=pd.DataFrame(scaled,columns=df_numeric.columns)

**The following thing to do, will be to encode the categorical variables**

In [45]:
encoder=OneHotEncoder(handle_unknown='error', drop='first')

In [46]:
encoder.fit(df_categorical1)

OneHotEncoder(drop='first')

In [47]:
df_cat_encoded=encoder.transform(df_categorical1).toarray()

In [48]:
df_cat_encoded.shape,df_num_scaled.shape

((95412, 24), (95412, 330))

In [49]:
X_concat=np.concatenate([df_num_scaled,df_cat_encoded], axis=1)

Now, we are going to define the "X" and the "y" in order to generate the test and train data.

In [50]:
y=df_tar['TARGET_B']
X=X_concat

In [51]:
y.value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [53]:
smote=SMOTE()

In [54]:
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
y_train_sm.value_counts()

1    63369
0    63369
Name: TARGET_B, dtype: int64

In [56]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_sm, y_train_sm)
model.score(X_test, y_test)

0.9500768585802124

In [57]:
y_pred = model.predict(X_test)

In [58]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     27200
           1       0.14      0.00      0.00      1424

    accuracy                           0.95     28624
   macro avg       0.55      0.50      0.49     28624
weighted avg       0.91      0.95      0.93     28624

